In [ ]:
!pip install -U relevanceai
!pip install tqdm>=4.62.2

In [ ]:
import pandas as pd

df = pd.read_csv('datasets/mpst_raw.csv')

df = df.sort_values('title').reset_index(drop=True)
df['lev'] = None
df
from Levenshtein import distance
for a in range(len(df)-1):
    if distance(df.iloc[a].title, df.iloc[a+1].title) <= 3:
        print(a, df.iloc[a].title, df.iloc[a+1].title)
        df.at[a, 'lev'] = distance(df.iloc[a].title, df.iloc[a+1].title)
df
#we filter similar movies
df = df[df['lev'].isnull()].reset_index(drop=True)
df

#find Avengers duplicates
for a in range(len(df)):
    if df.iloc[a]['title'].find('Avengers') != -1:
        pass
        #print(a)
#drop extra
df = df.drop([9572]).reset_index(drop=True) #i can do 1, 2, 3... to drop multiple
df
df.to_csv('mpst_no_duplicates.csv')

## encode with relevanceai

In [ ]:
json_files = df[['title', 'plot_synopsis']]
json_files = json_files.reset_index()
json_files.columns = ['_id', 'title', 'plot_synopsis']
json_files=json_files.to_dict(orient='records')
json_files

In [ ]:
#encode on local
from vectorhub.encoders.text.sentence_transformers import SentenceTransformer2Vec
model = SentenceTransformer2Vec("bert-base-uncased")
df_json = model.encode_documents(documents=json_files, fields=['plot_synopsis'])
df_json

## encode with sentence-transformers
An alternative to the relevanceai library

In [ ]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import numpy as np

tqdm.pandas()

model = SentenceTransformer('all-MiniLM-L6-v2') #all-MiniLM-L6-v2 #all-mpnet-base-v2
df_ = df.copy()
df_['plot_synopsis'] = df_['plot_synopsis'].progress_apply(lambda x : model.encode(x))
df_index = df_.pop('title')
df_ = df_[['plot_synopsis']]
df_ = pd.DataFrame(np.column_stack(list(zip(*df_.values))))
df_.index = df_index
df_
df_.to_csv('mpst_encoded_no_duplicates.csv')

# vector search

In [ ]:
import pandas as pd
df_movies_encoded = pd.read_csv('mpst_encoded_no_duplicates.csv')
df_movies_encoded.index = df_movies_encoded.pop('title')
df_movies_encoded

In [ ]:
from Levenshtein import distance
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd

nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(df_movies_encoded)

#string-searching algorithm
def closest_title(title):
    m = pd.DataFrame(df_movies_encoded.index)
    m['lev'] = m['title'].apply(lambda x : distance(x, 'Prince of Egypt'))
    return m.sort_values('lev', ascending=True)['title'].iloc[0]

In [ ]:
def find_similar_movies(df, nbrs, title):
    #if title not in df it will choose the best search
    title = closest_title(title)

    distances, indices = nbrs.kneighbors([df.loc[title]])

    #print(indices)
    #we print df data, no longer df_
    for index in indices[0][1:]:
        print('index', index)
        print(title, '->', df.iloc[index].name)

find_similar_movies(df_movies_encoded, nbrs, 'Prince of Egypt')